In [10]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
SEED=2020
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import classification_report
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Завантаження даних з CSV-файлів
users_activity = pd.read_csv("Users’ Activity.csv")
users = pd.read_csv("Users.csv")
streamers = pd.read_csv("Streamers.csv")


# Додавання префіксів до колонок
# Префікси для ознак користувачів
users.columns = ['user_' + col if col != 'userId' else 'userId' for col in users.columns]

# Префікси для ознак стрімерів
streamers.columns = ['streamer_' + col if col != 'streamerId' else 'streamerId' for col in streamers.columns]

# Префікси для ознак події (тільки ознаки, які є специфічними для подій)
users_activity = users_activity.rename(columns={
    'date': 'event_date',
    'timestamp': 'event_timestamp',
    'event': 'event_type',
    'streamId': 'event_streamId',
    'coins_spent_on_gift': 'event_coins_spent_on_gift'
})

# Створення цільової змінної
users_activity['event_sent_gift'] = users_activity['event_type'].apply(lambda x: 1 if x == 'user sent gift' else 0)

# Об'єднання даних за userId і streamerId
merged_data = pd.merge(users_activity, streamers, on='streamerId', how='left')
merged_data = pd.merge(merged_data, users, on='userId', how='left')

# Перетворюємо дату та часовий штамп у формат datetime
# Далі будемо використовувати дату інсталяції замість реєстрації через відсутність NaN у таких даних
merged_data['event_date'] = pd.to_datetime(merged_data['event_date'])
merged_data['event_timestamp'] = pd.to_datetime(merged_data['event_timestamp'])
# merged_data['user_registration_date'] = pd.to_datetime(merged_data['user_registration_date'])
# merged_data['streamer_registration_date'] = pd.to_datetime(merged_data['streamer_registration_date'])
merged_data['user_install_date'] = pd.to_datetime(merged_data['user_install_date'])
merged_data['streamer_install_date'] = pd.to_datetime(merged_data['streamer_install_date'])


# Обчислення кількості днів з дати інсталяції до дати події
# Кількість днів для користувачів
merged_data['days_since_user_install'] = (merged_data['event_date'] - merged_data['user_install_date']).dt.days

# Кількість днів для стрімерів
merged_data['days_since_streamer_install'] = (merged_data['event_date'] - merged_data['streamer_install_date']).dt.days
# Заміна 0 днів на 1 для уникнення ділення на 0
merged_data['days_since_streamer_install'] = merged_data['days_since_streamer_install'].replace(0, 1)

# Обчислення середніх значень коінів на день для стрімерів
merged_data['coins_per_day_streamer'] = merged_data['streamer_coins_earned_all_time'] / merged_data['days_since_streamer_install']


# Видалення первинних ознак

merged_data = merged_data.drop(columns=[
    'event_date',              # Дату події видаляємо після отримання нових ознак (строк існування стрімерів та користувачів)
    'event_timestamp',         # Часову мітку видаляємо після отримання нових ознак (строк існування стрімерів та користувачів)
    'event_type',              # Тип події не використовується для моделі, таргет вже сформовано. Інші типи подій, як-от 'user opened stream', 'user closed stream' тощо, не допоможуть у передбаченні того, чи відправить користувач подарунок.
    'event_streamId',          # Видаляємо event_streamId, оскільки це динамічна та недоступна заздалегідь ознака. Ми не можемо закладати в модель поведінку користувача поза стрімом, оскільки це невідомо на момент рекомендації.
    'event_coins_spent_on_gift', # Кількість витрачених коінів відома тільки після події
    'user_registration_date',  # Дата реєстрації користувача — видаляємо після розрахунку строку існування
    'user_install_date',       # Дата інсталяції користувача не потрібна - використовуємо дату реєстрації
    'streamer_registration_date', # Дата реєстрації стрімера — видаляємо після розрахунку строу існування
    'streamer_install_date'    # Дата інсталяції стрімера не потрібна - - використовуємо дату реєстрації
])

# Видаляємо всі рядки, де більше 8 значень NaN
merged_data = merged_data[merged_data.isna().sum(axis=1) <= 8]

# Коментар:
# Видаляємо всі рядки, де в понад 7 стовпцях є значення NaN.
# Це дозволяє позбутися неповних даних, де багато відсутніх значень, 
# що покращить якість даних для моделі.

# Припущення: У колонках user_followings/streamer_followers значення NaN означає, що користувач не має жодної підписки.
# Тому NaN замінюємо на 0, що відображатиме відсутність підписок.
merged_data['user_followings'] = merged_data['user_followings'].fillna(0)
merged_data['streamer_followers'] = merged_data['streamer_followers'].fillna(0)


# Використовуємо One-Hot Encoding для категоріальних ознак з невеликою кількістю унікальних значень і перетворюємо True/False у 1/0
data = pd.get_dummies(merged_data, columns=['streamer_gender', 'streamer_streamer_type', 'user_gender', 'user_media_source'])

# Перетворюємо всі значення True/False у 1/0
data = data.astype({col: 'int' for col in data.select_dtypes(include='bool').columns})

# Розділяємо дані на ознаки (X) та цільову змінну (y)
X = data.drop(columns=['event_sent_gift', 'userId', 'streamerId'])  # Вилучаємо айдішники та цільову змінну
y = data['event_sent_gift']  # Цільова змінна

# 1. СПОЧАТКУ РОБИМО TRAIN/TEST SPLIT (БЕЗ БУДЬ-ЯКОГО БАЛАНСУВАННЯ)
# Використовуємо stratify=y, щоб зберегти пропорцію класів у train та test.
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.1,
    random_state=SEED,
    stratify=y
)

# Прив’язуємо userId та streamerId до тренувальних та тестових рядків.
# Це ВАЖЛИВО: індекси X_train/X_test збігаються з індексами merged_data,
# тому ми точно зберігаємо відповідність user–streamer для кожного прикладу.
userId_train = merged_data.loc[X_train.index, 'userId']
userId_test  = merged_data.loc[X_test.index,  'userId']

streamerId_train = merged_data.loc[X_train.index, 'streamerId']
streamerId_test  = merged_data.loc[X_test.index,  'streamerId']


# 2. БАЛАНСУЄМО ЛИШЕ TRAIN-ЧАСТИНУ (це правильно — так уникаємо leakage)
X_train_majority = X_train[y_train == 0]
y_train_majority = y_train[y_train == 0]

X_train_minority = X_train[y_train == 1]
y_train_minority = y_train[y_train == 1]

# Downsampling мажоритарного класу під розмір міноритарного
X_majority_downsampled, y_majority_downsampled = resample(
    X_train_majority,
    y_train_majority,
    replace=False,                         # без повернення
    n_samples=len(y_train_minority),       # рівняємо кількість прикладів
    random_state=SEED
)

# Об’єднання збалансованих TRAIN-даних
X_train_bal = pd.concat([X_train_minority, X_majority_downsampled])
y_train_bal = pd.concat([y_train_minority, y_majority_downsampled])


# 3. НАВЧАЄМО МОДЕЛЬ НА ЗБАЛАНСОВАНОМУ TRAIN
xgb_clf = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=SEED
)

xgb_clf.fit(X_train_bal, y_train_bal)

# 4. РОБИМО ПРОГНОЗ НА ЧИСТОМУ TEST (без змін, без балансування)
y_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]

# Формуємо таблицю рекомендацій
recommendations = pd.DataFrame({
    'userId': userId_test,
    'streamerId': streamerId_test,
    'predicted_probability': y_pred_proba
})



In [11]:
# =============================
# 1. ЗНАЙДЕННЯ НАЙКРАЩОЇ ПАРИ
# =============================

# Знаходимо індекс рядка з максимальною передбаченою ймовірністю
best_pair_idx = recommendations['predicted_probability'].idxmax()

# Отримуємо найкращу пару user–streamer
best_pair = recommendations.loc[best_pair_idx]

print("Найкраща пара (userId, streamerId):")
print(best_pair)


Найкраща пара (userId, streamerId):
userId                   62ba70548ddfe12c02b2a97d
streamerId               630d28f9b5f77294876a4f88
predicted_probability                    0.999615
Name: 33828, dtype: object


In [12]:
# ================================================
# 2. ФУНКЦІЯ ДЛЯ ФОРМУВАННЯ ТОП-N СТРІМЕРІВ ДЛЯ ЮЗЕРА
# ================================================

def get_top_n_streamers_for_user(user_id, recommendations_df, N=10):
    """
    Повертає топ-N стрімерів для конкретного користувача user_id,
    відсортованих за ймовірністю predicted_probability.
    """
    # Фільтруємо рекомендації лише для вказаного користувача
    subset = recommendations_df[recommendations_df['userId'] == user_id]
    
    # Сортуємо за спаданням ймовірності
    subset_sorted = subset.sort_values(by='predicted_probability', ascending=False)
    
    # Повертаємо топ-N
    return subset_sorted.head(N)

u = userId_test.iloc[0]    # беремо будь-якого користувача з test
top10 = get_top_n_streamers_for_user(u, recommendations, N=10)
top10



,userId,streamerId,predicted_probability
306787,63387f4894c9c82f2a49c8d5,62bdcf83026081a7fb9196eb,0.751404
919853,63387f4894c9c82f2a49c8d5,62bd276421d28f8c1c44a75e,0.716293
622419,63387f4894c9c82f2a49c8d5,62bd276421d28f8c1c44a75e,0.652671
803958,63387f4894c9c82f2a49c8d5,6338c4ec94c9c82f2a49dedb,0.641840
804678,63387f4894c9c82f2a49c8d5,6338c4ec94c9c82f2a49dedb,0.641840
907526,63387f4894c9c82f2a49c8d5,6338263c94c9c82f2a49af21,0.495383
919560,63387f4894c9c82f2a49c8d5,6338efb908aa4f2581acc44a,0.468366
778805,63387f4894c9c82f2a49c8d5,62ace19731a8cf62a6a8959f,0.468307
778982,63387f4894c9c82f2a49c8d5,62ace19731a8cf62a6a8959f,0.468307
88464,63387f4894c9c82f2a49c8d5,633a35ef08aa4f2581ad2248,0.425206


In [13]:
# ============================================================================================
# 3. РАНДОМІЗАЦІЯ ФІДА ДЛЯ КОРИСТУВАЧА НА ОСНОВІ 5 КЛАСТЕРІВ СТРІМЕРІВ
# ============================================================================================

#------------------------------------------------------------------------------------------------
# 3.1. ПЕРЕРАХУНОК КЛАСТЕРІВ СТРІМЕРІВ (5 кластерів)
# ------------------------------------------------------------------------------------------------

# Групуємо стрімерів та обчислюємо середні значення числових ознак для кожного streamerId
streamers_unique = merged_data.groupby("streamerId").agg({
    'streamer_followers': 'mean',
    'streamer_coins_earned_all_time': 'mean',
    'streamer_avg_stream_duration': 'mean',
    'streamer_avg_watchers_per_stream': 'mean',
    'streamer_avg_earned_coins_per_stream': 'mean',
    'coins_per_day_streamer': 'mean',
    'days_since_streamer_install': 'mean'
}).reset_index()

# Масштабуємо числові ознаки перед кластеризацією
scaler = StandardScaler()
streamer_features_scaled = scaler.fit_transform(
    streamers_unique.drop(columns=['streamerId'])
)

# Запускаємо KMeans на 5 кластерах
kmeans = KMeans(n_clusters=5, random_state=SEED)
cluster_labels = kmeans.fit_predict(streamer_features_scaled)

# Додаємо номер кластера у таблицю
streamers_unique['cluster'] = cluster_labels

# Створюємо словник streamerId → номер кластера
streamer_cluster_map = streamers_unique.set_index("streamerId")['cluster']

print("Кластери стрімерів перераховано.")
display(streamers_unique.head())


# ------------------------------------------------------------------------------------------------
# 3.2. РАНДОМІЗАТОР ФІДА НА ОСНОВІ КЛАСТЕРІВ
# ------------------------------------------------------------------------------------------------

def randomize_feed_by_cluster(topN_df, cluster_map, top_fixed=3):
    """
    Рандомізує фід всередині кластерів:
    - перші top_fixed стрімерів залишаються статичними (найкращі рекомендації)
    - стрімери починаючи з позиції top_fixed перемішуються, але лише всередині свого кластера
    """

    df = topN_df.copy()
    
    # Додаємо колонку з номером кластера
    df['cluster'] = df['streamerId'].map(cluster_map)

    # Фіксована частина фіда (залишається без змін)
    fixed_part = df.head(top_fixed)

    # Частина, що підлягає рандомізації
    tail_part = df.iloc[top_fixed:]

    # Перемішуємо стрімерів у кожному кластері окремо
    randomized_tail = (
        tail_part
        .groupby('cluster', group_keys=False)
        .apply(lambda g: g.sample(frac=1))
        .reset_index(drop=True)
    )

    # З'єднуємо статичну та рандомізовану частину
    final_feed = pd.concat([fixed_part, randomized_tail], ignore_index=True)

    return final_feed


# ------------------------------------------------------------------------------------------------
# 3.3. ВИКЛИК РАНДОМІЗАЦІЇ ДЛЯ ВЖЕ ГОТОВОГО top10
# ------------------------------------------------------------------------------------------------

print("\nТоп-10 стрімерів (до рандомізації):")
display(top10)

final_feed = randomize_feed_by_cluster(top10, streamer_cluster_map, top_fixed=3)

print("\nФінальний фід (після рандомізації):")
display(final_feed)


Кластери стрімерів перераховано.


,streamerId,streamer_followers,streamer_coins_earned_all_time,streamer_avg_stream_duration,streamer_avg_watchers_per_stream,streamer_avg_earned_coins_per_stream,coins_per_day_streamer,days_since_streamer_install,cluster
0,61fa1880a2f894458fb71cfc,8271.0,41792.0,64.55,635.64,116.48,173.044527,241.513241,2
1,61fbb83fa2f89491e7b72b93,404.0,593.0,40.17,31.85,25.09,2.460581,241.000000,4
2,62064649c9b2860bd896d4ab,918.0,9115.0,2.59,10.13,22.49,38.952991,234.000000,4
3,6222007977aabb6c0a52b510,173.0,1960.0,1.54,7.25,13.57,9.245283,212.000000,4
4,6228d4fd77aabbf72952bb80,66.0,13.0,1.43,2.41,0.57,0.062500,208.000000,4



Топ-10 стрімерів (до рандомізації):


,userId,streamerId,predicted_probability
306787,63387f4894c9c82f2a49c8d5,62bdcf83026081a7fb9196eb,0.751404
919853,63387f4894c9c82f2a49c8d5,62bd276421d28f8c1c44a75e,0.716293
622419,63387f4894c9c82f2a49c8d5,62bd276421d28f8c1c44a75e,0.652671
803958,63387f4894c9c82f2a49c8d5,6338c4ec94c9c82f2a49dedb,0.641840
804678,63387f4894c9c82f2a49c8d5,6338c4ec94c9c82f2a49dedb,0.641840
907526,63387f4894c9c82f2a49c8d5,6338263c94c9c82f2a49af21,0.495383
919560,63387f4894c9c82f2a49c8d5,6338efb908aa4f2581acc44a,0.468366
778805,63387f4894c9c82f2a49c8d5,62ace19731a8cf62a6a8959f,0.468307
778982,63387f4894c9c82f2a49c8d5,62ace19731a8cf62a6a8959f,0.468307
88464,63387f4894c9c82f2a49c8d5,633a35ef08aa4f2581ad2248,0.425206



Фінальний фід (після рандомізації):


C:\Users\Dmitriy\AppData\Local\Temp\ipykernel_25272\685773503.py:66: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(frac=1))


,userId,streamerId,predicted_probability,cluster
0,63387f4894c9c82f2a49c8d5,62bdcf83026081a7fb9196eb,0.751404,4
1,63387f4894c9c82f2a49c8d5,62bd276421d28f8c1c44a75e,0.716293,1
2,63387f4894c9c82f2a49c8d5,62bd276421d28f8c1c44a75e,0.652671,1
3,63387f4894c9c82f2a49c8d5,6338c4ec94c9c82f2a49dedb,0.641840,0
4,63387f4894c9c82f2a49c8d5,6338c4ec94c9c82f2a49dedb,0.641840,0
5,63387f4894c9c82f2a49c8d5,633a35ef08aa4f2581ad2248,0.425206,0
6,63387f4894c9c82f2a49c8d5,6338263c94c9c82f2a49af21,0.495383,0
7,63387f4894c9c82f2a49c8d5,6338efb908aa4f2581acc44a,0.468366,2
8,63387f4894c9c82f2a49c8d5,62ace19731a8cf62a6a8959f,0.468307,2
9,63387f4894c9c82f2a49c8d5,62ace19731a8cf62a6a8959f,0.468307,2


#  Підсумки проекту

У цьому ноутбуці було виконано повний цикл побудови рекомендаційної системи для стрімерів:

## **1. Об’єднання та попередня обробка даних**
- проведено очищення, нормалізацію та формування цільової змінної;
- створено нові ознаки (days_since_install, coins_per_day тощо);
- виконано кодування категоріальних змінних;
- збережено індекси userId та streamerId для подальших рекомендацій.

## **2. Балансування та навчання моделі**
- реалізовано коректний train/test split з `stratify=y`;
- виконано балансування класів **лише на train-частині**, щоб уникнути leakage;
- навчено XGBoost-класіфікатор;
- отримано прогнозовані ймовірності взаємодії user → streamer.

## **3. Формування рекомендацій (Top-N)**
- реалізовано функцію отримання топ-N стрімерів для конкретного користувача;
- застосовано сортування за спаданням передбаченої ймовірності;
- побудовано базовий рекомендаційний список.

## **4. Кластеризація стрімерів**
- на основі числових ознак стрімерів обчислено **5 кластерів** (KMeans);
- сформовано мапу `streamerId → cluster`;
- кластеризація використовується для подальшої рандомізації фіда.

## **5. Рандомізація фіда в межах кластерів**
- перші *top-fixed* рекомендацій залишаються незмінними (найрелевантніші);
- інші стрімери перемішуються **лише всередині свого кластера**;
- забезпечено ефект "живого" та динамічного фіда без втрати релевантності.

---

### ✔ Результат
Отримано гнучку рекомендаційну систему, яка:
- визначає найкращих стрімерів для користувача;
- враховує поведінкову ймовірність взаємодії;
- забезпечує динамічний, оновлюваний фід за допомогою контрольованої рандомізації.

Система готова для подальшої інтеграції або розширення.
